In [2]:
!pip install pymysql

In [3]:
import pymysql
print(pymysql.__version__)

1.4.6


In [4]:
# ab_testing_pipeline.py
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [5]:
DB_USER = "root"
DB_PASS = "tanishasinha123"
DB_HOST = "localhost"
DB_NAME = "marketing"
DB_URI = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}?charset=utf8mb4"
engine = create_engine(DB_URI)

In [9]:
# ---------- Load data ----------
df = pd.read_sql("SELECT cu.user_id, cu.campaign_id, c.campaign_name, cu.converted, cu.revenue "
                 "FROM campaign_users cu JOIN campaigns c ON cu.campaign_id = c.campaign_id", engine)

In [10]:
# ---------- 1. Basic conversion rates ----------
summary = df.groupby("campaign_name")["converted"].agg(["count", "sum"])
summary["conversion_rate"] = summary["sum"] / summary["count"]
print(summary)

               count  sum  conversion_rate
campaign_name                             
Campaign A         2    1              0.5
Campaign B         2    1              0.5


In [11]:
# ---------- 2. Chi-square test ----------
contingency = pd.crosstab(df["campaign_name"], df["converted"])
chi2, p, dof, expected = chi2_contingency(contingency)
print("\nChi-square test:")
print("Chi2:", chi2, "p-value:", p)
if p < 0.05:
    print("=> Statistically significant difference in conversion rates")
else:
    print("=> No significant difference")


Chi-square test:
Chi2: 0.0 p-value: 1.0
=> No significant difference


In [12]:
# ---------- 3. Logistic regression ----------
# Prepare binary data
df["converted"] = df["converted"].astype(int)
df = pd.get_dummies(df, columns=["campaign_name"], drop_first=True)

In [14]:
df.columns = df.columns.str.replace(" ", "_")  # replace spaces with underscores

In [15]:
# Logistic regression: conversion ~ campaign indicator
model = smf.logit("converted ~ campaign_name_Campaign_B", data=df).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.693147
         Iterations 1
                           Logit Regression Results                           
Dep. Variable:              converted   No. Observations:                    4
Model:                          Logit   Df Residuals:                        2
Method:                           MLE   Df Model:                            1
Date:                Fri, 12 Sep 2025   Pseudo R-squ.:                   0.000
Time:                        11:36:58   Log-Likelihood:                -2.7726
converged:                       True   LL-Null:                       -2.7726
Covariance Type:            nonrobust   LLR p-value:                     1.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                                 0      1.414          

In [16]:
# Odds ratio
odds_ratio = np.exp(model.params)
print("\nOdds ratios:\n", odds_ratio)


Odds ratios:
 Intercept                           1.0
campaign_name_Campaign_B[T.True]    1.0
dtype: float64


In [17]:
# ---------- 4. ROI calculation ----------
roi = df.groupby("campaign_id").agg(
    total_revenue=("revenue","sum"),
    total_users=("user_id","count"),
    conversions=("converted","sum")
).reset_index()

In [18]:
budgets = pd.read_sql("SELECT campaign_id, budget FROM campaigns", engine)
roi = roi.merge(budgets, on="campaign_id")
roi["roi"] = (roi["total_revenue"] - roi["budget"]) / roi["budget"]
print("\nROI Table:\n", roi)


ROI Table:
    campaign_id  total_revenue  total_users  conversions   budget       roi
0            1           50.0            2            1  10000.0 -0.995000
1            2           70.0            2            1  12000.0 -0.994167
